# Some useful presentations

https://github.com/data-mining-in-action/DMIA_Sport_2019_Autumn/tree/master/lectures

# Create dataset with all unique combinations of given columns

In [ ]:
from itertools import product

index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in df['date_block_num'].unique():
    cur_shops = sales_train[sales_train['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales_train[sales_train['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales_train.groupby(index_cols, as_index=False).agg({'item_cnt_day': 'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num', 'shop_id', 'item_id'],inplace=True)

# Add calendar features

In [ ]:
import math
import calendar
import holidays
import datetime

date_blocks = pd.DataFrame(sales_train2.date_block_num.drop_duplicates())

def add_month(col):
    return int(divmod(col, 12)[1] + 1)

def add_year(col):
    return int(2013 + divmod(col+1, 12)[0])

def add_year_month(row):
    return f'{row.year}-{row.month}'

def add_season(col):
    if 0 < col <= 2 or col == 12:
        return 'winter'
    if 3 <= col < 6:
        return 'spring'
    if 6 <= col < 9:
        return 'summer'
    if 9 <= col < 12:
        return 'autumn'
    return np.nan

# get number of holidays + weekend in each month    
ru_holidays = []

for date, name in sorted(holidays.RU(years=[2013, 2014, 2015]).items()):
    ru_holidays.append(date)
    
def add_holidays_and_weekends(row):
    busines_dates = pd.bdate_range(f"{row.year}-{row.month}-01", f"{row.year}-{row.month}-{row.days_in_month}")
    busines_dates = [b for b in busines_dates if b not in ru_holidays]
    return row.days_in_month - len(busines_dates)
    
def add_weeks_in_month(row):
    for i in range(1, row.days_in_month+1):
        d = datetime.datetime(row.year, row.month, i)
        if d.day > d.weekday():
            startdate = d
            break
    return ((datetime.datetime(row.year, row.month, row.days_in_month) - startdate).days) //7 + 1 
    
def create_calendar_features(data):
    data['month'] = data.date_block_num.apply(add_month) 
    data['year'] = data.date_block_num.apply(add_year)
    data['year_month'] = data.apply(add_year_month, axis=1)
    data.year_month = pd.to_datetime(data.year_month, format='%Y-%m')
    data['days_in_month'] = data.year_month.dt.daysinmonth
    data['weeks_in_month'] = data.apply(add_weeks_in_month, axis=1)
    data['season'] = data.month.apply(add_season)
    data['holidays_and_weekends_in_month'] = data.apply(add_holidays_and_weekends, axis=1)


create_calendar_features(date_blocks)

# Create group features

In [ ]:
def create_gr_feats(data):
    # create aggregation feats for numeric features based on categorical ones
    for cat_col in ['shop_id', 'item_category_id', 'city', 'item_id']:
        for num_col in ['item_cnt_month', 'item_price']:
            for n, f in [('mean', np.mean), ('min', np.nanmin), ('max', np.nanmax)]:
                data[n + '_' + num_col + '_by_' + cat_col] = sales_train2.groupby(cat_col)[num_col].transform(f)
                
    # create features with counts
    for col in ['shop_id', 'item_id', 'item_category_id', 'city', 'global_category', 'item_price']:
        data[col + '_cnt'] = data[col].map(sales_train2[col].value_counts(dropna = False))

# Replace rare values with "RARE VALUE" string

In [ ]:
def create_col_with_min_freq(data, col, min_freq=10):
    # replace rare values (less than min_freq rows) in feature by RARE_VALUE
    data[col + '_fixed'] = data[col].astype(str)
    data.loc[sales_train2[col].value_counts()[data[col + '_fixed']].values < min_freq, col + 
             '_fixed'] = "RARE_VALUE"


# Get data from previous month(s) for timeseries prediction

In [ ]:
mean_cols = [c for c in sales_train2.columns if c.startswith('mean_item_cnt', 0, 13)] + ['item_cnt_month']

def create_time_features():
    global sales_train2, sales_test2
    win = [i for i in range(1, 4)]
    for w in win:
        tmp = sales_train2[['date_block_num', 'shop_id', 'item_id'] + mean_cols].copy()
        tmp.date_block_num = tmp.date_block_num + w
        for c in mean_cols:
            tmp.rename({c: c + str(w)}, axis=1, inplace=True)

        sales_train2 = sales_train2.join(tmp.set_index(['date_block_num', 'shop_id', 'item_id']), 
                                         on=['date_block_num', 'shop_id', 'item_id'])
        sales_test2 = sales_test2.join(tmp.set_index(['date_block_num', 'shop_id', 'item_id']), 
                                       on=['date_block_num', 'shop_id', 'item_id'])
        del tmp
        gc.collect()

# Define if text is English

In [ ]:
def isEnglish(s):
    ss = "ª°⭐•®’—–™&\xa0\xad\xe2\xf0"  # special characters
    s = str(s).lower()
    for k in range(len(ss)):
        s = s.replace(ss[k], "")
    try:
        s.encode(encoding="utf-8").decode("ascii")
    except UnicodeDecodeError:
        # not english; check it still not english if western european characters are removed
        ss = "éáñóüäýöçãõúíàêôūâşè"
        for k in range(len(ss)):
            s = s.replace(ss[k], "")
        try:
            s.encode(encoding="utf-8").decode("ascii")
        except UnicodeDecodeError:
            return 3  # really not english
        else:
            return 2  # spanish/french?
    else:
        return 1  # english

# Convert japaneese alphabet

In [ ]:
def convert_japanese_alphabet(df):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except Exception:
                pass
        return row

    df = df.apply(convert, axis=1)
    return df

# Process some categorical like features (couple of categories in one string)

In [ ]:
def process_categories(cat, split=" "):
    cat = [x for x in str(cat).split(split) if cat != "" and len(x) >= 2]
    # Keep only letters
    cat = [re.sub(r"[^a-zA-Z]", " ", x) for x in cat]
    # Delete multi space
    cat = [re.sub("\\s+", " ", x).strip() for x in cat]
    return cat

# Function to fill missing categories
def find_cat(name, Key_words_for_cat):
    name_list = process_categories(unidecode(str(name).lower()))
    for cat, wordlist in Key_words_for_cat.items():
        if any(name_word in name_list for name_word in wordlist):
            return cat
    return ""

def process_text_cat(text):
    text = unidecode(text.lower())
    res = " ".join([re.sub(r"[^a-zA-Z]", " ", x).strip() for x in text.split()])
    return re.sub("\\s+", " ", res).strip()


def simplify_cat(categories, Cat_regroup):
    categories = str(categories).lower()
    if categories in ("", "nan"):
        return -1
    for cat in categories.split(","):
        cat = process_text_cat(cat)
        for i, Liste in enumerate(Cat_regroup):
            if any(cat == x for x in Liste):
                return i + 1
    else:
        return 0

# Set all features combinations (brute force)

In [ ]:
import itertools

features = df.drop(['f_1', 'target'], axis=1).columns
generated_features = pd.DataFrame()

# with all of these features usually will work too long, so try to comment some of them 
# and select features separately
for fe_a, fe_b in itertools.combinations(features, 2):

    generated_features[f'{fe_a}+{fe_b}']   = df[fe_a] + df[fe_b]
    generated_features[f'{fe_a}-{fe_b}']   = df[fe_a] - df[fe_b] 
    generated_features[f'{fe_a}*{fe_b}']   = df[fe_a] * df[fe_b]
    generated_features[f'{fe_a}/{fe_b}']   = df[fe_a] / df[fe_b]

    generated_features[f'{fe_a}*{fe_b}_2'] = df[fe_a] * df[fe_b].pow(2)
    generated_features[f'{fe_a}_2*{fe_b}'] = df[fe_a].pow(2) * df[fe_b]
    generated_features[f'{fe_a}_2']        = df[fe_a].pow(2)
    generated_features[f'{fe_b}_2']        = df[fe_b].pow(2)

    generated_features[f'{fe_a}_05'] = df[fe_a].pow(0.5)
    generated_features[f'{fe_b}_05'] = df[fe_b].pow(0.5)
    generated_features[f'{fe_a}*{fe_b}_05'] = df[fe_a] * df[fe_b].pow(0.5)
    generated_features[f'{fe_a}_05*{fe_b}'] = df[fe_a].pow(0.5) * df[fe_b]
    
    generated_features[f'{fe_a}_log'] = np.log(df[fe_a])
    generated_features[f'{fe_b}_log'] = np.log(df[fe_b])
    generated_features[f'{fe_a}*{fe_b}_log'] = df[fe_a] * np.log(df[fe_b])
    generated_features[f'{fe_a}_log*{fe_b}'] = np.log(df[fe_a]) * df[fe_b]

# Create features with PolynomialFeatures

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# train set
poly_train = PolynomialFeatures()
data_transform = df.drop('Class', axis=1).copy()
data_poly = pd.DataFrame(poly_train.fit_transform(data_transform),
                         columns=poly_train.get_feature_names_out(), index=data_transform.index)
data_poly = data_poly.drop('1', axis=1)
data_poly = pd.concat([data_poly, df[['Class']]], axis=1)
data_poly['Class'] = data_poly.Class.astype('int64')

# test set
poly_test = PolynomialFeatures()
test_poly = pd.DataFrame(poly_test.fit_transform(test_df), columns=poly_test.get_feature_names_out())
test_poly = test_poly.drop('1', axis=1)

# Create feature interaction using trees